In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle
import sys

In [2]:
links = []
for page_num in tqdm(range(615)):
    response = requests.get(f"https://alfahir.hu/kereso?kereses=%20&from=2019-01-01&to=2019-12-31&field_authors&field_tags&page={page_num}")
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = soup.find_all("div", class_="field field-name-node-title")
    links.extend(["https://alfahir.hu" + title.find("a").get("href") for title in titles])
# writing to file
with open('links.p', 'wb') as f:
    pickle.dump(links, f)

In [2]:
# reading from file
with open('links.p', 'rb') as f:
    links = pickle.load(f)

In [15]:
alfa_dict = {}
for url in tqdm(links[:50]):
    related_urls = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    date_time = soup.find("div", class_="article-dates").text.replace("\n", "")
    body = soup.find("div", class_="field field--name-body field--type-text-with-summary field--label-hidden field--item")
    paragraphs = body.find_all("p")
    for paragraph in paragraphs:
        if "MTI" in paragraph.text or "mti" in paragraph.text or "Mti" in paragraph.text:
            related_urls.append("MTI")
        if paragraph.find_all("a") is not None:
            for link in paragraph.find_all("a"):
                if "http" in link.get('href'):
                    related_urls.append(link.get('href'))
                else:
                    related_urls.append("https:" + link.get('href'))
    alfa_dict[url] = {
    "related_links" : related_urls,
    "publish_date" : date_time
}

In [16]:
alfa_dict

{'https://alfahir.hu/2019/12/31/tuzijatek_balatonalmadi_polgarmester_allatbarat': {'related_links': [],
  'publish_date': '2019. december 31. | 21:30'},
 'https://alfahir.hu/2019/12/31/gyilkossag_kobanya_elfogas_edesanya': {'related_links': ['https://index.hu/belfold/2019/12/31/kobanya_emberoles_vercseppek_harmat_utca_bantalmazas/'],
  'publish_date': '2019. december 31. | 21:13'},
 'https://alfahir.hu/2019/12/31/kepek_szilveszter_buek_20192020': {'related_links': [],
  'publish_date': '2019. december 31. | 18:33'},
 'https://alfahir.hu/2019/12/31/egyesult_allamok_kina_kereskedelmi_haboru_enyhules': {'related_links': ['MTI'],
  'publish_date': '2019. december 31. | 17:42'},
 'https://alfahir.hu/2019/12/31/emmi_szinhazak_tagadas_tamogatas': {'related_links': ['MTI'],
  'publish_date': '2019. december 31. | 16:41'},
 'https://alfahir.hu/2019/12/31/tuzolto_okf_tuzijatek_szilveszter': {'related_links': ['https://www.katasztrofavedelem.hu/29/hirek/225567/hogyan-ne-szilveszterezzunk-a-tuzolt